In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
data = pd.read_csv("order_brush_order.csv")
data["event_time"] = pd.to_datetime(data["event_time"])
data.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [3]:
finalList = pd.DataFrame(columns=["shopid", "userid"])

In [5]:
for shopid, group in data.groupby(["shopid"]):
    usrList = pd.Series()
    for index, row in group.iterrows():
        st = row.event_time
        unit = group[(st <= group['event_time']) & (group['event_time'] < st + datetime.timedelta(hours=1))]
        trsAmt = unit.shape[0]
        UsrAmt = len(unit["userid"].unique())
        if (trsAmt/UsrAmt) >= 3:
            temp = unit["userid"].value_counts()
            usrList = usrList.append(pd.Series(temp[temp.values == max(temp)].index))
    blackList = ""
    usrList = usrList.drop_duplicates().sort_values()
    for i, v in usrList.iteritems():
        blackList = blackList + "&" + str(v)
    if blackList == "":
        blackList = "0"
    else:
        blackList = blackList[1:]
    finalList =finalList.append([{"shopid": shopid,"userid": blackList}],ignore_index=True)
finalList.to_csv("./submission.csv", index = False)
finalList